In [1]:
import altair as alt
import pandas as pd
import numpy as np
import streamlit as st
from vega_datasets import data

In [53]:
#TEMPLATE TO GET TOTAL SINGLE FILES - stored in sub_datasets
df1 = pd.read_csv('datasets/us_states_covid19_daily.csv')

columns_to_show = ['positive', 'negative', 'hospitalizedCurrently','hospitalizedCumulative','state']
df2 = df1.groupby(["state"])[columns_to_show].agg([np.max])

total_positive = df2[['state', 'positive']].reset_index()
total_positive.columns = ['State', 'drop', 'positive']
total_positive = total_positive[['State', 'positive']]
total_positive.index = np.arange(1, len(total_positive) + 1) #Makes index start from 1 instead of 0
total_positive.to_csv('total_positive.csv')

In [54]:
#CHOROPLETH TEMPLATE
states = alt.topo_feature(data.us_10m.url, 'states')

ch = pd.read_csv('sub_datasets/total_positive.csv')
ch.columns = ['Index', 'State', 'Positive']

#Creates the CH map
ch_map = alt.Chart(states).mark_geoshape().encode(
    color='Positive:Q',
    tooltip=['id:O', 'Positive:Q']
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(ch, 'Index', ['Positive'])
).project(
    type='albersUsa'
).properties(
    width=600,
    height=400
)

In [55]:
#ZIPPING FILE COMMAND: zip -r example.zip original_folder